In [48]:
import urllib.request
import json
import pymongo
import pandas as pd
import datetime

In [49]:
 # get the bbc rss feed of news stories and connect to it
earthquake_url = "http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/significant_month.geojson"

In [50]:
def save_earthquakes():
    try:
        response = urllib.request.urlopen(earthquake_url)
    except urllib.error.URLError as e:
        if hasattr(e, 'reason'):
            print('We failed to reach a server.')
            print('Reason: ', e.reason)
        elif hasattr(e, 'code'):
            print('The server couldn\'t fulfill the request.')
            print('Error code: ', e.code)
    else:
        # the url request was successful - convert the response to a string
        json_string = response.read().decode('utf-8')

        # the json package loads() converts the string to python dictionaries and lists
        eq_json = json.loads(json_string)

        # from the json dictionary we get the title to print
        title = eq_json['metadata']['title']
        print('Collected data from', title)
        #  and we get the list of earthquakes
        quakelist = eq_json['features']

        # Connection to Mongo DB
        try:
            client=pymongo.MongoClient('localhost', 27017)
            print ("Connected successfully!!!")
        except pymongo.errors.ConnectionFailure as e:
           print ("Could not connect to MongoDB: %s" % e )
        else:

            # use database named usgs or create it if not there already
            eqdb = client.usgs
            # create collection named earthquakes or create it if not there already
            quakecoll = eqdb.earthquakes
            # add all the earthquakes to the list
            quakecoll.insert_many(quakelist)
            print("Added", len(quakelist), "to earthquakes collection in usgs database")
            # close the database connection
        client.close()

In [51]:
def get_earthquakes():
    quake_docs = []
    # Connection to Mongo DB
    try:
        client=pymongo.MongoClient('localhost', 27017)
        print ("Connected successfully!!!")
    except pymongo.errors.ConnectionFailure as e:
        print ("Could not connect to MongoDB: %s" % e )
    else:
        # use database named usgs or create it if not there already
        eqdb = client.usgs
        # create collection named earthquakes or create it if not there already
        quakecoll = eqdb.earthquakes
        # add all the earthquakes to the list
        for doc in quakecoll.find():
            place = doc["properties"]["place"]
            # grab unix timestamp in milliseconds
            unix_time_mil = doc["properties"]["time"]
            # convert to unix in seconds
            unix_time = unix_time_mil / 1000
            quake_docs.append([place,unix_time])
            
        # close the database connection
    client.close()
    quake_docs = pd.DataFrame(quake_docs)
    quake_docs.columns = ['Location','Time']
    quake_docs['Time'] = quake_docs.Time.apply(lambda unix_time: datetime.datetime.fromtimestamp(unix_time).isoformat())
    return quake_docs

In [52]:
def list_earthquakes():
    quake_docs = get_earthquakes()
    print(quake_docs)

In [47]:
list_earthquakes()

Connected successfully!!!
                              Location                        Time
0             16km SE of Su'ao, Taiwan  2019-08-07T17:28:02.548000
1             54km ENE of Namie, Japan  2019-08-04T06:23:03.862000
2   102km WSW of Tugu Hilir, Indonesia  2019-08-02T08:03:27.483000
3        94km SW of San Antonio, Chile  2019-08-01T14:28:05.073000
4          62km E of Lakatoro, Vanuatu  2019-07-31T11:02:33.851000
5   24km S of La Libertad, El Salvador  2019-07-31T01:54:55.289000
6           140km ESE of Shingu, Japan  2019-07-27T14:31:07.514000
7       16km S of Twentynine Palms, CA  2019-07-22T12:26:56.080000
8           3km NNW of Magoula, Greece  2019-07-19T07:13:16.136000
9                12km WSW of Byron, CA  2019-07-16T16:11:01.470000
10   36km SSE of Kendalrejo, Indonesia  2019-07-15T20:18:36.991000
11            16km SE of Su'ao, Taiwan  2019-08-07T17:28:02.548000
12            54km ENE of Namie, Japan  2019-08-04T06:23:03.862000
13  102km WSW of Tugu Hilir, Indones